In [2]:
!pip install requests --quiet --upgrade
!pip install beautifulsoup4 --quiet --upgrade


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json
import boto3
import logging
from botocore.exceptions import ClientError

In [5]:
stock_exchange = 'NASDAQ'
stock_name = 'AMZN'

In [6]:
home_url =f'https://eoddata.com/stockquote/{stock_exchange}/' 
stock_url = f'{home_url}{stock_name}.htm'
stock_url


'https://eoddata.com/stockquote/NASDAQ/AMZN.htm'

In [7]:
response = requests.get(stock_url)      #requests.get()
print(response.status_code)

200


In [8]:
page_contents = response.text
len(page_contents)    #The `len` fucnction tells us the length of the response object

59573

In [9]:
doc = BeautifulSoup(page_contents, 'html.parser')  #Now 'doc' contains entire html in parsed format
len(doc)

5

In [10]:
title = doc.find('title')
title.string

'\r\n\tNASDAQ, AMZN End of Day and Historical Stock Data [Amazon.com Inc]\r\n'

In [11]:
tabels = doc.find_all('table')
table_rows = tabels[7].find_all('tr')

In [31]:
headers = table_rows[0]
keys = [i.string.lower() for i in headers.find_all('th')]
keys = keys[0:-1]
keys

['date', 'open', 'high', 'low', 'close', 'volume']

In [13]:
table_rows[1]

<tr><td>07/05/24</td><td align="right">198.7</td><td align="right">200.6</td><td align="right">198.2</td><td align="right">200.0</td><td align="right">39,858,800</td><td align="right">0</td></tr>

In [39]:
data = table_rows[1].find_all('td')
values = [i.string for i in data]
values = values[0:-1]
values

['07/05/24', '198.7', '200.6', '198.2', '200.0', '39,858,800']

In [47]:
data = dict(zip(keys,values))
data

{'date': '07/05/24',
 'open': '198.7',
 'high': '200.6',
 'low': '198.2',
 'close': '200.0',
 'volume': '39,858,800'}

In [48]:
#Change the type of the columns
data['date'] = datetime.strptime(data['date'], '%m/%d/%y').strftime('%Y-%m-%d')
data['open'] = float(data['open'])
data['high'] = float(data['high'])
data['low'] = float(data['low'])
data['close'] = float(data['close'])
data['volume'] = int(data['volume'].replace(',', ''))

In [49]:
data

{'date': '2024-07-05',
 'open': 198.7,
 'high': 200.6,
 'low': 198.2,
 'close': 200.0,
 'volume': 39858800}

In [52]:
def scrape(stock,exchange):
    home_url =f'https://eoddata.com/stockquote/{exchange}/'   #The URL Address of the webpage we will scrape, i.e. Stocks starting from A
    stock_url = f'{home_url}{stock}.htm'

    response = requests.get(stock_url)      #requests.get()
#     print(response.status_code)

    page_contents = response.text

    doc = BeautifulSoup(page_contents, 'html.parser')  #Now 'doc' contains entire html in parsed format

    tabels = doc.find_all('table')
    table_rows = tabels[7].find_all('tr')
    
    headers = table_rows[0]
    keys = [i.string.lower() for i in headers.find_all('th')]
    keys = keys[0:-1]
    
    row = table_rows[1].find_all('td')
    values = [i.string for i in row]
    values = values[0:-1]

    data = dict(zip(keys,values))
    
    data['date'] = datetime.strptime(data['date'], '%m/%d/%y').strftime('%Y-%m-%d')
    data['open'] = float(data['open'])
    data['high'] = float(data['high'])
    data['low'] = float(data['low'])
    data['close'] = float(data['close'])
    data['volume'] = int(data['volume'].replace(',', ''))
    
    return(data)
    
    

In [53]:
stock = 'AMZN'
exchange = 'NASDAQ'
scrape(stock,exchange)

{'date': '2024-07-08',
 'open': 200.0,
 'high': 201.2,
 'low': 198.0,
 'close': 198.9,
 'volume': 25789691}

In [56]:
import os
path = os.getcwd()
file_path = f"{path}\\data\\stock_latest.json"
# print(file_path)


bucket = 's3-datalake-stock-price-darshil0208'
object_name = 'stock_latest_price.json'
stock = 'AMZN'
exchange = 'NASDAQ'


In [58]:
def create_json(data):
    date = data['date']
    # print(len(data))
    # print(url)
    with open(file_path,'w') as f:
        json.dump(data,f,separators=(',', ':'))
    return

create_json(scrape(stock,exchange))

In [59]:
def upload_file_to_s3(file_name,bucket,object_name):
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_path, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [60]:
if upload_file_to_s3(file_path,bucket,object_name):
    print("Upload successful")
else:
    print("Upload Unsuccessful")

Upload successful
